In [49]:
%matplotlib inline
import sys
sys.path.append('data/')
import tables as tb
import numpy as np
import pandas as pd
from datetime import datetime
import scipy.sparse as sp
import matplotlib.pyplot as plt
import sqlite3 as lite
from model import DateMapper, Minute

### 일별 OHLC 변환하여 TDOP (Trade Density Over Price) 만들기 

In [107]:
db = tb.open_file('data/market.hdf5', mode='a')

In [108]:
#종목정보 불러오기
con = lite.connect('data/db.sqlite3')
products = pd.read_sql('select * from trading_product', con)
products.set_index(['group'], drop=False, inplace=True)
products = products.to_dict(orient='index')

In [112]:
#테이블 삭제후 생성
for group in db.iter_nodes('/'):
    #group.DateMapper.remove()
    #group.Minute.remove()
    db.create_table(group, 'DateMapper', DateMapper)
    db.create_table(group, 'Minute', Minute)

In [113]:
for group in db.iter_nodes('/'):
    grpname = group._v_name
    name = group._v_title
    print(group._v_name)

    data = []
    dates = []
    #db cursors
    datecur = group.DateMapper 
    minute = group.Minute
        
    #환경변수
    tick_unit = products[grpname]['tick_unit']
    digit = products[grpname]['decimal_places']
        
    #pandas dataframe
    df = pd.DataFrame(group.Daily.read()).sort_values('date', ascending=True)
    
    for row in df.itertuples():
        idx = len(dates) #datemapper
        
        if round(row.low, digit) == round(row.high, digit):
            item = (idx, round(row.low, digit), row.volume)
            dateitem = (row.date, idx)
            data.append(item)
            dates.append(dateitem)
        
        else:
            length = (row.high - row.low)/tick_unit + 1
            length = np.rint(length)
            value = row.volume/length
            
            if np.isinf(value) or (value < 0.1): #inf 또는 틱탕 너무 작은 value 버림
                print("wrong volume", row.volume, length, name, str(row.date.astype('M8[s]').astype('M8[D]')))
                continue
            else:
                dateitem = (row.date, idx)
                dates.append(dateitem)
                for price in np.arange(round(row.low, digit), row.high-tick_unit/2, tick_unit):
                    item = (idx, price, value)
                    data.append(item)
    if dates:
        datecur.append(dates)
        minute.append(data)
        minute.flush()
        datecur.flush()
print("Everythin fin.")

AD
BP
BR
wrong volume 10 101.0 Brazilian Real 2009-01-07
wrong volume 4 117.0 Brazilian Real 2009-01-23
wrong volume 6 77.0 Brazilian Real 2009-03-02
wrong volume 5 106.0 Brazilian Real 2009-06-09
wrong volume 9 237.0 Brazilian Real 2009-08-11
wrong volume 11 174.0 Brazilian Real 2009-08-14
wrong volume 1 127.0 Brazilian Real 2009-08-19
wrong volume 4 47.0 Brazilian Real 2009-11-10
wrong volume 4 151.0 Brazilian Real 2009-11-13
wrong volume 4 137.0 Brazilian Real 2010-05-14
wrong volume 1 158.0 Brazilian Real 2010-06-15
wrong volume 14 201.0 Brazilian Real 2011-05-31
wrong volume 2 43.0 Brazilian Real 2011-06-03
wrong volume 4 131.0 Brazilian Real 2011-06-15
wrong volume 1 116.0 Brazilian Real 2011-10-21
wrong volume 12 170.0 Brazilian Real 2011-12-01
wrong volume 1 245.0 Brazilian Real 2011-12-08
wrong volume 1 174.0 Brazilian Real 2011-12-12
wrong volume 3 72.0 Brazilian Real 2011-12-14
wrong volume 1 76.0 Brazilian Real 2011-12-22
wrong volume 2 112.0 Brazilian Real 2012-01-12
wrong

wrong volume 4 61.0 ICE Cotton 2007-05-08
wrong volume 3 336.0 ICE Cotton 2007-07-02
wrong volume 1 76.0 ICE Cotton 2007-07-06
wrong volume 1 151.0 ICE Cotton 2007-07-09
wrong volume 2 101.0 ICE Cotton 2007-10-01
wrong volume 4 176.0 ICE Cotton 2007-10-02
wrong volume 2 104.0 ICE Cotton 2007-10-05
wrong volume 24 1305.0 ICE Cotton 2008-03-03
wrong volume 12 226.0 ICE Cotton 2008-03-05
wrong volume 6 119.0 ICE Cotton 2008-04-29
wrong volume 3 72.0 ICE Cotton 2008-07-03
wrong volume 3 59.0 ICE Cotton 2008-09-26
wrong volume 8 126.0 ICE Cotton 2008-09-30
wrong volume 6 392.0 ICE Cotton 2008-10-01
wrong volume 1 32.0 ICE Cotton 2008-10-02
wrong volume 5 160.0 ICE Cotton 2008-10-06
wrong volume 8 261.0 ICE Cotton 2008-10-07
wrong volume 12 186.0 ICE Cotton 2008-11-28
wrong volume 11 121.0 ICE Cotton 2008-12-02
wrong volume 14 387.0 ICE Cotton 2008-12-04
wrong volume 18 369.0 ICE Cotton 2008-12-05
wrong volume 9 98.0 ICE Cotton 2009-02-27
wrong volume 6 111.0 ICE Cotton 2009-03-02
wrong volu

wrong volume 3 34.0 ICE ORANGE JUICE 2002-12-23
wrong volume 3 41.0 ICE ORANGE JUICE 2003-07-10
wrong volume 8 111.0 ICE ORANGE JUICE 2007-09-07
wrong volume 17 250.0 ICE ORANGE JUICE 2007-11-08
wrong volume 12 121.0 ICE ORANGE JUICE 2008-01-09
wrong volume 3 31.0 ICE ORANGE JUICE 2008-03-07
wrong volume 4 56.0 ICE ORANGE JUICE 2008-03-10
wrong volume 3 100.0 ICE ORANGE JUICE 2008-05-02
wrong volume 1 16.0 ICE ORANGE JUICE 2008-05-05
wrong volume 2 26.0 ICE ORANGE JUICE 2008-07-09
wrong volume 1 59.0 ICE ORANGE JUICE 2008-11-06
wrong volume 10 268.0 ICE ORANGE JUICE 2009-07-06
PA
wrong volume 2 61.0 Palladium 1988-02-04
wrong volume 1 30.0 Palladium 1988-02-08
wrong volume 1 22.0 Palladium 1988-02-18
wrong volume 3 61.0 Palladium 1988-03-24
wrong volume 1 50.0 Palladium 1988-04-13
wrong volume 1 11.0 Palladium 1988-04-20
wrong volume 2 26.0 Palladium 1988-04-27
wrong volume 1 29.0 Palladium 1988-05-20
wrong volume 1 30.0 Palladium 1988-07-28
wrong volume 2 30.0 Palladium 1989-03-08
wro

wrong volume 2 22.0 Platinum 1995-07-24
wrong volume 1 30.0 Platinum 1995-08-02
wrong volume 1 28.0 Platinum 1996-01-24
wrong volume 2 29.0 Platinum 1996-04-24
wrong volume 1 23.0 Platinum 1996-07-12
wrong volume 1 14.0 Platinum 1996-07-17
wrong volume 2 31.0 Platinum 1997-01-21
wrong volume 1 55.0 Platinum 1997-02-25
wrong volume 2 34.0 Platinum 1997-04-14
wrong volume 9 109.0 Platinum 1997-07-18
wrong volume 1 46.0 Platinum 1997-10-24
wrong volume 2 75.0 Platinum 1998-01-16
wrong volume 2 73.0 Platinum 1998-01-23
wrong volume 1 20.0 Platinum 1998-04-21
wrong volume 2 91.0 Platinum 1998-04-27
wrong volume 2 43.0 Platinum 1998-06-08
wrong volume 1 14.0 Platinum 1998-07-13
wrong volume 1 46.0 Platinum 1998-07-22
wrong volume 1 21.0 Platinum 1998-10-15
wrong volume 1 19.0 Platinum 1999-04-14
wrong volume 4 44.0 Platinum 1999-07-08
wrong volume 1 31.0 Platinum 1999-07-13
wrong volume 7 81.0 Platinum 1999-07-26
wrong volume 1 107.0 Platinum 1999-10-07
wrong volume 4 61.0 Platinum 1999-10-2

SI
wrong volume 1 15.0 Silver 1988-01-12
wrong volume 1 30.0 Silver 1988-06-28
wrong volume 2 26.0 Silver 1994-01-31
wrong volume 2 24.0 Silver 1994-06-27
wrong volume 1 19.0 Silver 1995-01-23
wrong volume 1 11.0 Silver 1996-04-08
wrong volume 2 30.0 Silver 1996-06-04
wrong volume 2 49.0 Silver 1998-02-18
wrong volume 2 76.0 Silver 1998-06-17
wrong volume 4 46.0 Silver 1998-06-19
wrong volume 1 23.0 Silver 1998-09-30
wrong volume 1 23.0 Silver 2001-12-05
wrong volume 1 19.0 Silver 2001-12-21
wrong volume 1 17.0 Silver 2003-06-13
wrong volume 2 32.0 Silver 2004-02-09
wrong volume 1 14.0 Silver 2004-04-01
wrong volume 2 54.0 Silver 2005-01-03
wrong volume 1 40.0 Silver 2005-10-14
wrong volume 1 11.0 Silver 2005-10-18
wrong volume 1 58.0 Silver 2006-01-10
wrong volume 2 41.0 Silver 2006-02-13
wrong volume 5 53.0 Silver 2006-04-18
wrong volume 2 24.0 Silver 2006-04-25
wrong volume 7 75.0 Silver 2006-09-19
wrong volume 2 61.0 Silver 2007-01-09
wrong volume 2 56.0 Silver 2007-04-02
wrong vol

### Verification

In [118]:
# 중복데이터 체크
from collections import Counter

for grp in db.iter_nodes('/'):
    item = grp.DateMapper.cols.date[:]
    dup = [item for item, count in Counter(df2).items() if count > 1]
    if dup:
        print(grp._v_title, " has duplicated data")